In [1]:
import os
import gzip
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [3]:
!mkdir data

In [4]:
!mv /Users/priyaltripathi/Downloads/GSE235063_RAW.tar /Users/priyaltripathi/scRNA_and_scATAC_analysis/data

In [5]:
!tar -xf /Users/priyaltripathi/scRNA_and_scATAC_analysis/data/GSE235063_RAW.tar

In [6]:
!mv GSM* data/

In [7]:
os.listdir("data")

['GSM7494271_AML7_DX_processed_matrix.mtx.gz',
 'GSM7494316_AML25_DX_processed_metadata.tsv.gz',
 'GSM7494327_AML12_REL_raw_genes.tsv.gz',
 'GSM7494264_AML2_REL_raw_genes.tsv.gz',
 'GSM7494302_AML22_REM_processed_metadata.tsv.gz',
 'GSM7494288_AML27_REM_processed_genes.tsv.gz',
 'GSM7494286_AML27_DX_raw_genes.tsv.gz',
 'GSM7494314_AML14_DX_raw_genes.tsv.gz',
 'GSM7494266_AML15_DX_processed_metadata.tsv.gz',
 'GSM7494308_AML24_REM_processed_metadata.tsv.gz',
 'GSM7494318_AML25_REM_raw_matrix.mtx.gz',
 'GSM7494269_AML3_DX_processed_matrix.mtx.gz',
 'GSM7494321_AML26_REM_processed_metadata.tsv.gz',
 'GSM7494310_AML23_REL_processed_barcodes.tsv.gz',
 'GSM7494321_AML26_REM_processed_genes.tsv.gz',
 'GSM7494278_AML20_REM_raw_genes.tsv.gz',
 'GSM7494313_AML28_REM_processed_barcodes.tsv.gz',
 'GSM7494260_AML6_DX_processed_matrix.mtx.gz',
 'GSM7494267_AML15_REL_raw_barcodes.tsv.gz',
 'GSM7494264_AML2_REL_raw_barcodes.tsv.gz',
 'GSM7494318_AML25_REM_raw_genes.tsv.gz',
 'GSM7494286_AML27_DX_proce

In [3]:
files = os.listdir('data/')

In [12]:
ffs = [x for x in files if 'features' in x]

In [13]:
ffs

['GSM7494282_AML17_DX_raw_features.tsv.gz',
 'GSM7494291_AML9_REM_raw_features.tsv.gz',
 'GSM7494267_AML15_REL_processed_features.tsv.gz',
 'GSM7494326_AML12_DX_processed_features.tsv.gz',
 'GSM7494328_AML12_REM_processed_features.tsv.gz',
 'GSM7494298_AML4_DX_raw_features.tsv.gz',
 'GSM7494315_AML14_REM_processed_features.tsv.gz',
 'GSM7494312_AML28_REL_processed_features.tsv.gz',
 'GSM7494304_AML21_REL_processed_features.tsv.gz',
 'GSM7494278_AML20_REM_raw_features.tsv.gz',
 'GSM7494309_AML23_DX_processed_features.tsv.gz',
 'GSM7494261_AML6_REL_raw_features.tsv.gz',
 'GSM7494265_AML2_REM_processed_features.tsv.gz',
 'GSM7494312_AML28_REL_raw_features.tsv.gz',
 'GSM7494257_AML16_DX_processed_features.tsv.gz',
 'GSM7494281_AML5_REM_processed_features.tsv.gz',
 'GSM7494311_AML23_REM_processed_features.tsv.gz',
 'GSM7494285_AML1_DX_raw_features.tsv.gz',
 'GSM7494301_AML22_REL_raw_features.tsv.gz',
 'GSM7494279_AML5_DX_raw_features.tsv.gz',
 'GSM7494281_AML5_REM_raw_features.tsv.gz',
 'GS

In [14]:
!mkdir temp

In [15]:
for i in ffs:
    with gzip.open('data/' + i, 'rt') as f_in:
        with gzip.open('temp/' + i, 'wt') as f_out:
            for line in f_in:
                f_out.write(line.strip() +  "\tGene Expression\n")

In [17]:
!mv temp/* data/

In [19]:
!mkdir raw_adata

In [4]:
# List all files in the 'data/' directory excluding hidden files, 'processed', and 'tar' files
files = [x for x in os.listdir('data/') if not x.startswith('.') and 'processed' not in x and 'tar' not in x]

# Loop through unique prefixes in the files
for prefix in set([x.split('_raw')[0] + '_raw_' for x in files]):
    # Read the 10x mtx data
    adata = sc.read_10x_mtx('data/', prefix=prefix)
    
    # Filter out cells with less than 2000 UMIs (to remove low-quality cells)
    sc.pp.filter_cells(adata, min_counts=2000)
    
    # Perform random sampling to reduce the number of cells further (e.g., 50% of remaining cells)
    adata = adata[adata.obs.sample(frac=0.5).index, :].copy()
    
    # Write the filtered and reduced dataset to an .h5ad file
    adata.write_h5ad('raw_adata/' + prefix + '.h5ad')